<a href="https://colab.research.google.com/github/fajariqs/stress-analysis-ibm-granite/blob/main/Fajar_LLM_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install replicate
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [12]:
from google.colab import userdata
import os

# ambil api_token dari secret
api_token = userdata.get("api_token")

# lempar api_token ke env. var.
os.environ["REPLICATE_API_TOKEN"] = api_token

In [13]:
from langchain_community.llms import Replicate

# Get the LLM Ai
llm = Replicate(model="ibm-granite/granite-3.3-8b-instruct")

In [14]:
output = llm.invoke("who is messi?")

print(output)

Lionel Messi is an Argentine professional footballer who plays as a forward for the Primera División club Rosario Central and captains the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or trophies and ten European Golden Shoes. He is the record goalscorer for Argentina and La Liga, and has scored over 700 senior career goals for club and country. Messi led Argentina to victory in the 2021 Copa América, securing his first major international title.


In [7]:
!pip install pandas
!pip install -U langchain langchain-community langchain-experimental replicate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 2.6 MB/s eta 0:00:00


In [17]:
import pandas as pd

df = pd.read_csv("StressLevelDataset.csv")
df

,anxiety_level,self_esteem,mental_health_history,depression,headache,blood_pressure,sleep_quality,breathing_problem,noise_level,living_conditions,...,basic_needs,academic_performance,study_load,teacher_student_relationship,future_career_concerns,social_support,peer_pressure,extracurricular_activities,bullying,stress_level
0,14,20,0,11,2,1,2,4,2,3,...,2,3,2,3,3,2,3,3,2,1
1,15,8,1,15,5,3,1,4,3,1,...,2,1,4,1,5,1,4,5,5,2
2,12,18,1,14,2,1,2,2,2,2,...,2,2,3,3,2,2,3,2,2,1
3,16,12,1,15,4,3,1,3,4,2,...,2,2,4,1,4,1,4,4,5,2
4,16,28,0,7,2,3,5,1,3,2,...,3,4,3,1,2,1,5,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,11,17,0,14,3,1,3,2,2,2,...,3,2,2,2,3,3,2,3,3,1
1096,9,12,0,8,0,3,0,0,0,1,...,4,0,1,1,1,1,3,4,3,2
1097,4,26,0,3,1,2,5,2,2,3,...,4,5,1,4,1,3,1,2,1,0
1098,21,0,1,19,5,3,1,4,3,1,...,1,2,5,1,4,1,4,4,4,2


In [19]:
from langchain_experimental.agents.agent_toolkits.pandas.base import (
    create_pandas_dataframe_agent
)

agent = create_pandas_dataframe_agent(
    llm=Replicate(model="ibm-granite/granite-3.3-8b-instruct"),
    df=df,
    verbose=True,
    allow_dangerous_code=True,  # opt-in for Python REPL execution
)

In [42]:
df['stress_level'].unique()

array([1, 2, 0])

In [43]:
# mapping kategori
# kolom stress_level hanya punya nilai 0, 1, dan 2, bukan 1–5 seperti asumsi awal.
# pas dipanggil stress_levels[3], stress_levels[4], atau stress_levels[5] muncul KeyError.

stress_levels = df['stress_level'].value_counts().to_dict()


result_df = pd.DataFrame(
    {
        'Stress Level': ['Low', 'Normal', 'High'],
        'Count': [
            stress_levels.get(0, 0),  # Low
            stress_levels.get(1, 0),  # Normal
            stress_levels.get(2, 0)   # High
        ]
    }
)

print(result_df)


  Stress Level  Count
0          Low    373
1       Normal    358
2         High    369


In [44]:
low = stress_levels.get(0, 0)
normal = stress_levels.get(1, 0)
high = stress_levels.get(2, 0)

summary = f"""
Dari dataset:
- Low = {low}
- Normal = {normal}
- High = {high}

Distribusi menunjukkan stress cenderung rendah–sedang,
dengan hanya sedikit kasus tingkat tinggi.
"""
print(summary)



Dari dataset: 
- Low = 373 
- Normal = 358 
- High = 369

Distribusi menunjukkan stress cenderung rendah–sedang, 
dengan hanya sedikit kasus tingkat tinggi.



In [46]:
agent.invoke({
    "input": """
Classify the dataset based on the column 'stress_level'.
Use this mapping:
- 0 = Low
- 1 = Normal
- 2 = High

Output only the results in a concise table with two columns: 'Stress Level' and 'Count'.

"""})



> Entering new AgentExecutor chain...
Thought: To classify the dataset based on the 'stress_level' column and count the occurrences for each stress level, I will use the `value_counts()` function on the 'stress_level' column. This function will return a series with stress levels as the index and their respective counts as values. I will then format this series into a table with 'Stress Level' and 'Count' columns.

Action: python_repl_ast

Action Input: 
```python
stress_counts = df['stress_level'].value_counts()
stress_counts.rename(index={0: 'Low', 1: 'Normal', 2: 'High'}, inplace=True)
stress_counts
```
stress_level
Low       373
High      369
Normal    358
Name: count, dtype: int64Final Answer:

| Stress Level | Count |
|--------------|-------|
|      Low     |  373  |
|    Normal    |  358  |
|      High    |  369  |

Final Answer:

| Stress Level | Count |
|--------------|-------|
|      Low     |  373  |
|    Normal    |  358  |
|      High    |  369  |

> Finished chain.


{'input': "\nClassify the dataset based on the column 'stress_level'.\nUse this mapping:\n- 0 = Low\n- 1 = Normal\n- 2 = High\n\nOutput only the results in a concise table with two columns: 'Stress Level' and 'Count'.\n\n",
 'output': '| Stress Level | Count |\n|--------------|-------|\n|      Low     |  373  |\n|    Normal    |  358  |\n|      High    |  369  |'}

In [47]:
agent.invoke({
    "input": """
Summarize the dataset based on the column 'stress_level'.
Use this mapping:
- 0 = Low
- 1 = Normal
- 2 = High

Your summary should include:
1. The distribution of Low, Normal, and High stress levels.
2. Key insights (e.g., which category dominates, any missing categories, potential implications).
3. Keep the explanation concise and clear.
"""
})



> Entering new AgentExecutor chain...
Thought: To summarize the dataset based on the 'stress_level' column using the given mapping, I will use pandas' value_counts method to count occurrences for each stress level category (0, 1, 2). Then, I will interpret these counts to provide insights.

Action: python_repl_ast

Action Input:
```python
stress_level_counts = df['stress_level'].value_counts().to_dict()
```
Thought: I have obtained the counts for each stress level category. Now, I will map these counts to the descriptions (Low, Normal, High) and summarize the distribution.

Action: python_repl_ast

Action Input:
```python
stress_level_summary = {
    'Low': stress_level_counts.get(0, 0),
    'Normal': stress_level_counts.get(1, 0),
    'High': stress_level_counts.get(2, 0)
}
```
I now know the final answer
Final Answer: 

1. **Distribution of Stress Levels:**
   - Low (0): stress_level_summary['Low']
   - Normal (1): stress_level_summary['Normal']
   - High (2): stress_level_summary[

{'input': "\nSummarize the dataset based on the column 'stress_level'.\nUse this mapping:\n- 0 = Low\n- 1 = Normal\n- 2 = High\n\nYour summary should include:\n1. The distribution of Low, Normal, and High stress levels.\n2. Key insights (e.g., which category dominates, any missing categories, potential implications).\n3. Keep the explanation concise and clear.\n",
 'output': "1. **Distribution of Stress Levels:**\n   - Low (0): stress_level_summary['Low']\n   - Normal (1): stress_level_summary['Normal']\n   - High (2): stress_level_summary['High']\n\n2. **Key Insights:**\n   - From the counts obtained, we can determine which category dominates. For instance, if 'Low' stress level has the highest count, it suggests that most individuals in the dataset have low stress.\n   - We can check for any missing categories based on our mapping (0, 1, 2). If a category is consistently zero, it might imply missing data or an error in the dataset.\n   - Potential implications can be inferred based o

In [49]:
agent.invoke({
    "input": """
Classify and summarize the dataset based on the column 'stress_level'.
Use this mapping:
- 0 = Low
- 1 = Normal
- 2 = High

Instructions:
1. Count the occurrences of each category and create a DataFrame with two columns: 'Stress Level' and 'Count'.
2. Do not reference any undefined variables when creating the DataFrame (e.g., do not use summary_df.columns).
3. After the table, provide a short summary with:
   - The distribution of Low, Normal, and High stress levels (with percentages).
   - Key insights, such as which category dominates, whether some categories are missing, and possible implications.

Output should include both the classification table and the summary.
"""
})




> Entering new AgentExecutor chain...
Thought: To address the question, I need to categorize the 'stress_level' column into 'Low', 'Normal', and 'High' based on the provided mapping. Then, I'll count the occurrences of each category and summarize the distribution along with key insights.

Action: python_repl_ast

Action Input:
```python
stress_levels = df['stress_level'].map({0: 'Low', 1: 'Normal', 2: 'High'})
stress_counts = stress_levels.value_counts().reset_index()
stress_counts.columns = ['Stress Level', 'Count']
stress_counts
```
  Stress Level  Count
0          Low    373
1         High    369
2       Normal    358I have categorized and counted the occurrences of each stress level. Now, I will summarize the distribution along with key insights.

Action: python_repl_ast

Action Input:
```python
total_count = df.shape[0]
low_percentage = (stress_counts[stress_counts['Stress Level'] == 'Low']['Count'].values[0] / total_count) * 100
normal_percentage = (stress_counts[stress_counts[

{'input': "\nClassify and summarize the dataset based on the column 'stress_level'.\nUse this mapping:\n- 0 = Low\n- 1 = Normal\n- 2 = High\n\nInstructions:\n1. Count the occurrences of each category and create a DataFrame with two columns: 'Stress Level' and 'Count'.\n2. Do not reference any undefined variables when creating the DataFrame (e.g., do not use summary_df.columns).\n3. After the table, provide a short summary with:\n   - The distribution of Low, Normal, and High stress levels (with percentages).\n   - Key insights, such as which category dominates, whether some categories are missing, and possible implications.\n\nOutput should include both the classification table and the summary.\n",
 'output': "### Classification Table\n\n| Stress Level | Count |\n|--------------|-------|\n|     Low      |  373  |\n|   Normal     |  358  |\n|    High      |  369  |\n\n### Summary\n\n**Distribution of Stress Levels:**\n\n- Low stress levels: 37.3% (373 instances)\n- Normal stress levels: